In [ ]:
from obspy import UTCDateTime
from obspy.taup import TauPyModel
import pandas as pd
from rtergpy_reference.run import defaults, event, etime2name
from rtergpy_reference.waveforms import getwaves
from AttenuationFunctionsTesting import (
    check_existing_dataframe, processANSStxt, filteringst,
    maxamp_calc_freq_bands, max_and_normalized_max_plots)

# Initialize TauPyModel for seismic wave travel time calculations
model = TauPyModel(model="iasp91")

# Load configuration and event defaults
Defaults = defaults()
Event = event()
Defaults.src = 'RASPISHAKE'
Defaults.network = 'AM'
Defaults.chan = 'EHZ'
Defaults.stationrange = [0.5, 10.0]
Event.ecount = '00'
Event.newData = False  # Use already downloaded data

# Process or load ANSS event data
#ANSS = processANSStxt() #If you have raw ANSS list, process with this
ANSS = pd.read_csv('ANSS_processed_data.csv')  # comment out top if already processed to load with this
check_existing_dataframe()  # Check for existing DataFrame to append results

edateold = ""

for index, EQ in ANSS.iterrows():
    eloc = [EQ.Latitude, EQ.Longitude, EQ.Depth]
    magnitude = {'type': EQ.Mtype, 'value': EQ.Mag}
    year, mo, dy = EQ.Date.split('-')
    hh, mn, sec = EQ.Time.split(':')
    etime = UTCDateTime(int(year), int(mo), int(dy), int(hh), int(mn), float(sec))

    Event.ecount = str(int(Event.ecount) + 1).zfill(2) if EQ.Date == edateold else '00'
    Event.iter = 'RS' if Defaults.src == 'RASPISHAKE' else 'IRIS'
    edateold = EQ.Date
    Event.eventname = etime2name(etime, ecount=Event.ecount) + Event.iter
    Event.origin = [eloc, etime]

    print(f"\n\n{Event.eventname} ===============================")

    try:
        st, df = getwaves(Defaults, Event)
        print("Completed getwaves")
        stp = filteringst(st)
        maxamp_calc_freq_bands(stp, EQ, Defaults, etime, eloc)
    except Exception as e:
        print(f"ERROR: {Event.eventname} could not complete getwaves due to {e}")